In [71]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et
import re

pd.options.display.max_columns = 999

# Florida

## VEST Documentation

Election results from the Florida Department of State (https://dos.myflorida.com/elections/data-statistics/elections-data/precinct-level-election-results/)

Precinct results from several sources.  
- Alachua, Bay, Bradford, Brevard, Calhoun, Citrus, Clay, Dixie, Escambia, Hardee, Hendry, Hernando, Indian River, Lafayette and Sarasota come from the Department of State.  
- Broward, Gadsden, Gilchrist, Manatee, Nassau, Santa Rosa, St. Johns, St. Lucie, and Union are from the Orlando Sentinel's precinct map for the 2016 presidential primary (http://interactive.orlandosentinel.com/elections/2016/presidential-primary/results/dem.html).  
- Baker, Charlotte, Franklin, Glades, Holmes, Jackson, Jefferson, Levy, Liberty, Madison, Monroe, Okeechobee, Orange, Seminole, Suwannee, Taylor, Wakulla, Walton, and Washinton come from the U.S. Census Bureau's 2020 Redistricting Data Program.  
- Collier, Columbia, DeSoto, Duval, Flagler, Gulf, Highlands, Hillsborough, Lake, Lee, Leon, Marion, Martin, Miami-Dade, Okaloosa, Osceola, Palm Beach, Pasco, Pinellas, Polk, Putnam, Sumter, and Volusia come from the counties.
- Hamilton was drawn based on a geocoded voter registration file.  
- Edits were made to Brevard, Charlotte, Escambia, Franklin, Indian River, Jefferson, Lafayette, Lake, and Lee, Miami-Dade, and St. Johns based on the voter registration file.  

Lake 108, Osceola 999, Palm Beach 8002, Seminole 900s, and Monroe's "Cumulative" precinct (the latter appears on the county's detailed results but not the DOS's precinct results file) don't have geography. Brevard 999, Broward Z073, Collier 450, Flagler 999, Hillborough 999, Leon 9000, Miami-Dade 100, and Pinellas 512 do have geography, but they are just the county election offices. Both groups represent some type of vote not assigned to a particular geography, like UOCAVA results, so these were distributed across the county by candidate proportional to the vote each precinct recorded.

G20PRERTRU - Donald J. Trump (Republican Party)  
G20PREDBID - Joseph R. Biden (Democratic Party)  
G20PRELJOR - Jo Jorgensen (Libertarian Party)  
G20PREODEL - Roque "Rocky" De La Fuente (Reform Party)  
G20PRESLAR - Gloria La Riva (Party for Socialism and Liberation)  
G20PREGHAW - Howie Hawkin (Green Party)  
G20PRECBLA - Don Blankenship (Constitution Party)  
G20PREOWRI - Write-in Votes  

## Load VEST File

In [72]:
vest_fl_20 = gp.read_file("./raw-from-source/VEST/fl_2020/fl_2020.shp")

In [73]:
data_columns = [col for col in vest_fl_20.columns if "G20" in col]

## Load Election Results

Fields and codes data https://files.floridados.gov/media/694099/precinct-level-results-data-definition-field-codes.pdfF

In [74]:
#This Data is from the FL department of state, and can only be downloaded county by county
all_files = os.listdir("./raw-from-source/Election_Results/2020-general-election-rev/")

In [75]:
#Minor processing note: For the FRA file, I replaced ""AJ"" w/ "AJ", so the load works

In [76]:
#Check that all files have the same number of columns
for i in all_files:
    if i !=".DS_Store":
        ref = "./raw-from-source/Election_Results/2020-general-election-rev/"
        file_ref = ref+i
        file_prev = pd.read_csv(file_ref,sep="\t",engine='python',index_col=None, header=None)
        if (file_prev.shape[1]!=19):
            print(i)
        #print(file_prev.shape)
    
#All the files have 19 columns, so they should be good to combine

In [77]:
li = []
for i in all_files:
    if i not in [".DS_Store","DAD_PctResults20201103recount.txt","BRO_PctResults20201103recount.txt"]:
        ref = "./raw-from-source/Election_Results/2020-general-election-rev/"
        file_ref = ref+i
        file_prev = pd.read_csv(file_ref,sep="\t",engine='python',index_col=None, header=None)
        li.append(file_prev)
frame = pd.concat(li, axis=0, ignore_index=True)
print(frame.shape)

(553307, 19)


https://files.floridados.gov/media/694099/precinct-level-results-data-definition-field-codes.pdf

In [78]:
col_rename_dict = {0: 'County Code (Three-character abbreviation)',
 1: 'County Name',
 2: 'Election Number',
 3: 'Election Date',
 4: 'Election Name',
 5: 'Unique Precinct Identifier',
 6: 'Precinct Polling Location',
 7: 'Total Registered Voters',
 8: 'Total Registered Republicans',
 9: 'Total Registered Democrats',
 10: 'Total Registered All Other Parties',
 11: 'Contest Name',
 12: 'District',
 13: 'Contest Code (Florida’s 6 digit contest codes)',
 14: 'Candidate/Retention/IssueName/WriteInsCast/OverVotes/UnderVotes',
 15: 'Candidate Party (abbreviation)',
 16: 'Candidate Florida Voter Registration System ID Number',
 17: 'DOE Assigned Candidate Number or Retention/Issue Number',
 18: 'Vote Total'}

In [79]:
frame.rename(columns = col_rename_dict, inplace = True)

In [80]:
races_list = ['President of the United States', 'Representative in Congress',
       'State Representative','State Senator','Retention of Carlos G. Muñiz','Amendment No. 1: Citizenship Requirement to Vote in Florida Elections',
       'Amendment No. 2: Raising Florida’s Minimum Wage',
       'Amendment No. 3: All Voters Vote in Primary Elections for State Legislature, Governor, and Cabinet',
       'Amendment No. 4: Voter Approval of Constitutional Amendments',
       'Amendment No. 5: Limitations on Homestead Property Tax Assessments; increased portability period to transfer accrued benefit',
       'Amendment No. 6: Ad Valorem Tax Discount for Spouses of Certain Deceased Veterans Who Had Permanent, Combat-Related Disabilities']

In [81]:
#Filter to the presidential results
frame =frame[frame['Contest Name'].isin(races_list)]

#Filter out the OverVotes and UnderVotes
filtered_frame = frame[~frame['Candidate/Retention/IssueName/WriteInsCast/OverVotes/UnderVotes'].isin(['OverVotes', 'UnderVotes'])]

#Clean up the precinct column
filtered_frame['Precinct Polling Location'] = filtered_frame['Precinct Polling Location'].astype(str)

#Deal with Dade County
filtered_frame["mod_col"]=filtered_frame.apply(lambda row:int(float(row['Precinct Polling Location'].split("PRECINCT ")[1])) if row['County Code (Three-character abbreviation)']=="DAD" else row['Unique Precinct Identifier'], axis=1)

#Make the precinct column at least 4 digits
filtered_frame["mod_col"]=filtered_frame["mod_col"].astype(str)
filtered_frame["modified_pre"]=filtered_frame["mod_col"].str.zfill(4)

#Make a column with the 3 letter county code and the precincts
filtered_frame["pct_std"]=filtered_frame['County Code (Three-character abbreviation)']+filtered_frame["modified_pre"]

<ipython-input-81-5edbeda0f2eb>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame['Precinct Polling Location'] = filtered_frame['Precinct Polling Location'].astype(str)
<ipython-input-81-5edbeda0f2eb>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame["mod_col"]=filtered_frame.apply(lambda row:int(float(row['Precinct Polling Location'].split("PRECINCT ")[1])) if row['County Code (Three-character abbreviation)']=="DAD" else row['Unique Precinct Identifier'], axis=1)
<ipython-input

In [82]:
# Cast the contest name and code columns to string to make the pivot col
filtered_frame["Contest Name"] = filtered_frame["Contest Name"].astype(str)
filtered_frame["Contest Code (Florida’s 6 digit contest codes)"] = filtered_frame["Candidate/Retention/IssueName/WriteInsCast/OverVotes/UnderVotes"].astype(str)

<ipython-input-82-46da4f0e8365>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame["Contest Name"] = filtered_frame["Contest Name"].astype(str)
<ipython-input-82-46da4f0e8365>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame["Contest Code (Florida’s 6 digit contest codes)"] = filtered_frame["Candidate/Retention/IssueName/WriteInsCast/OverVotes/UnderVotes"].astype(str)


In [83]:
filtered_frame['Candidate Party (abbreviation)'] = filtered_frame['Candidate Party (abbreviation)'].map({' ':'WRI'}).fillna(filtered_frame['Candidate Party (abbreviation)'])

<ipython-input-83-cc08b28a7227>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame['Candidate Party (abbreviation)'] = filtered_frame['Candidate Party (abbreviation)'].map({' ':'WRI'}).fillna(filtered_frame['Candidate Party (abbreviation)'])


In [84]:
filtered_frame['Candidate Party (abbreviation)'].unique()

array(['REP', 'DEM', 'LPF', 'REF', 'PSL', 'GRE', 'CPF', 'WRI', 'NPA',
       'NOP'], dtype=object)

In [85]:
# Create the pivot column with the contest name and the candidate
filtered_frame["pivot_col"] = filtered_frame["Contest Name"] + "-" + filtered_frame["Contest Code (Florida’s 6 digit contest codes)"] + "-" + "PARTY:" +filtered_frame['Candidate Party (abbreviation)']  

<ipython-input-85-fe582c8917a9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame["pivot_col"] = filtered_frame["Contest Name"] + "-" + filtered_frame["Contest Code (Florida’s 6 digit contest codes)"] + "-" + "PARTY:" +filtered_frame['Candidate Party (abbreviation)']


In [86]:
# Where it exists, add in the district to the pivot column
filtered_frame["pivot_col"] = np.where(filtered_frame["District"]!=" ",filtered_frame["pivot_col"] + "-" + filtered_frame["District"], filtered_frame["pivot_col"])

<ipython-input-86-d12f4a46ad53>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame["pivot_col"] = np.where(filtered_frame["District"]!=" ",filtered_frame["pivot_col"] + "-" + filtered_frame["District"], filtered_frame["pivot_col"])


### Pivot the Data

In [87]:
filtered_frame[filtered_frame["Contest Name"]== "Representative in Congress"]

,County Code (Three-character abbreviation),County Name,Election Number,Election Date,Election Name,Unique Precinct Identifier,Precinct Polling Location,Total Registered Voters,Total Registered Republicans,Total Registered Democrats,Total Registered All Other Parties,Contest Name,District,Contest Code (Florida’s 6 digit contest codes),Candidate/Retention/IssueName/WriteInsCast/OverVotes/UnderVotes,Candidate Party (abbreviation),Candidate Florida Voter Registration System ID Number,DOE Assigned Candidate Number or Retention/Issue Number,Vote Total,mod_col,modified_pre,pct_std,pivot_col
110,OKE,Okeechobee,10866,11/03/2020,2020 General Election,1,1-ABIDING HOPE WORSHIP CENTER,1593,0,0,0,Representative in Congress,District 17,Greg Steube,Greg Steube,REP,105486919,75229,929,1,0001,OKE0001,Representative in Congress-Greg Steube-PARTY:R...
111,OKE,Okeechobee,10866,11/03/2020,2020 General Election,2,2-BASINGER COMMUNITY CENTER,849,0,0,0,Representative in Congress,District 17,Greg Steube,Greg Steube,REP,105486919,75229,515,2,0002,OKE0002,Representative in Congress-Greg Steube-PARTY:R...
112,OKE,Okeechobee,10866,11/03/2020,2020 General Election,3,3-OAKVIEW BAPTIST CHURCH,2449,0,0,0,Representative in Congress,District 17,Greg Steube,Greg Steube,REP,105486919,75229,1527,3,0003,OKE0003,Representative in Congress-Greg Steube-PARTY:R...
113,OKE,Okeechobee,10866,11/03/2020,2020 General Election,4,4-OKEECHOBEE CIVIC CENTER,3021,0,0,0,Representative in Congress,District 17,Greg Steube,Greg Steube,REP,105486919,75229,1031,4,0004,OKE0004,Representative in Congress-Greg Steube-PARTY:R...
114,OKE,Okeechobee,10866,11/03/2020,2020 General Election,5,5-FREEDOM RANCH,1567,0,0,0,Representative in Congress,District 17,Greg Steube,Greg Steube,REP,105486919,75229,807,5,0005,OKE0005,Representative in Congress-Greg Steube-PARTY:R...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551503,GAD,Gadsden,10866,11/03/2020,2020 General Election,21,Pre 21 Green Shade AME Church,566,0,0,0,Representative in Congress,District 5,Al Lawson,Al Lawson,DEM,104987227,75494,266,21,0021,GAD0021,Representative in Congress-Al Lawson-PARTY:DEM...
551504,GAD,Gadsden,10866,11/03/2020,2020 General Election,22,Pre 22 Second Elizabeth Church,711,0,0,0,Representative in Congress,District 5,Al Lawson,Al Lawson,DEM,104987227,75494,424,22,0022,GAD0022,Representative in Congress-Al Lawson-PARTY:DEM...
551505,GAD,Gadsden,10866,11/03/2020,2020 General Election,23,Pre 23 New Direction Christian,787,0,0,0,Representative in Congress,District 5,Al Lawson,Al Lawson,DEM,104987227,75494,397,23,0023,GAD0023,Representative in Congress-Al Lawson-PARTY:DEM...
551506,GAD,Gadsden,10866,11/03/2020,2020 General Election,24,Pre 24 St. Joseph Missionary B,477,0,0,0,Representative in Congress,District 5,Al Lawson,Al Lawson,DEM,104987227,75494,347,24,0024,GAD0024,Representative in Congress-Al Lawson-PARTY:DEM...


In [88]:
# Removing all of the specific party registration data, as it looks like the state doesn't have that
for val in ['Total Registered Republicans','Total Registered Democrats', 'Total Registered All Other Parties']:
    print(filtered_frame[val].unique())

[0]
[0]
[0]


In [89]:
filtered_frame.drop(['Election Number', 'Election Date', 'Total Registered Republicans','Total Registered Democrats', 'Total Registered All Other Parties','Candidate Florida Voter Registration System ID Number','DOE Assigned Candidate Number or Retention/Issue Number'], axis = 1, inplace = True)

/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [90]:
#Pivot the data so that each row has all the results from that precinct
pivoted_2020 = pd.pivot_table(filtered_frame, values=['Vote Total'], index=["pct_std",'County Code (Three-character abbreviation)','County Name','Total Registered Voters'],columns=['pivot_col'],aggfunc=sum)

#Clean up the indices
pivoted_2020.reset_index(inplace=True,drop=False)

In [91]:
pivoted_2020.columns

MultiIndex([(                                   'pct_std', ...),
            ('County Code (Three-character abbreviation)', ...),
            (                               'County Name', ...),
            (                   'Total Registered Voters', ...),
            (                                'Vote Total', ...),
            (                                'Vote Total', ...),
            (                                'Vote Total', ...),
            (                                'Vote Total', ...),
            (                                'Vote Total', ...),
            (                                'Vote Total', ...),
            ...
            (                                'Vote Total', ...),
            (                                'Vote Total', ...),
            (                                'Vote Total', ...),
            (                                'Vote Total', ...),
            (                                'Vote Total', ...),
         

In [92]:
pivoted_2020.columns = [' '.join(col).strip() for col in pivoted_2020.columns.values]

In [93]:
pivoted_2020.columns = pivoted_2020.columns.str.replace("Vote Total ","")

## Rename Columns

In [94]:
keep_names = ['pct_std', 'County Code (Three-character abbreviation)', 'County Name','Total Registered Voters']

In [95]:
def get_race(contest):
    if "President" in contest:
        level = "PRE"
    elif ("Representative in Congress" in contest or "State Senator" in contest or "State Representative" in contest):
        contest_info = get_level_dist(contest)
        level = contest_info[0]+contest_info[1]        
    else:
        print(contest)
        raise ValueError
    return level

In [96]:
def get_level_dist(column_name):
    zfill_level = 2
    if "Representative in Congress" in column_name:
        level = "CON"
    elif "State Senator" in column_name:
        level = "SU"
    elif "State Representative" in column_name:
        level = "SL"
        zfill_level = 3
    else:
        raise ValueError
    return_val = re.findall("District \S*",column_name) 
    if (len(return_val)!=0):
        dist = return_val[0].split(" ")[1]
        dist = dist.zfill(zfill_level)
    else:
        raise ValueError
    return level,dist

In [97]:
def get_party(contest):
    if "PARTY:DEM" in contest:
        return "D"
    elif "PARTY:REP" in contest:
        return "R"
    elif "PARTY:LPF" in contest:
        return "L"
    ## Reform -> F
    elif "PARTY:REF" in contest:
        return "O"
    elif "PARTY:PSL" in contest:
        return "S"
    elif "PARTY:GRE" in contest:
        return "G"
    elif "PARTY:CPF" in contest:
        return "C"
    elif "PARTY:WRI" in contest:
        return "O"
    elif "PARTY:NPA" or "PARTY:NOP" in contest:
        return "N"
    else:
        print(contest)
        return ValueError

In [98]:
def get_name(contest):
    contest = contest.upper()
    name = contest.split("-")[1]
    likely_last = name.split(" ")[-1]
    if likely_last in ["JR","III","II","SR"]:
        return name.split(" ")[-2][0:3]
    else:
        return likely_last[0:3]

In [99]:
races_list = ['President of the United States', 'Representative in Congress',
       'State Representative','State Senator','Retention of Carlos G. Muñiz','Amendment No. 1: Citizenship Requirement to Vote in Florida Elections',
       'Amendment No. 2: Raising Florida’s Minimum Wage',
       'Amendment No. 3: All Voters Vote in Primary Elections for State Legislature, Governor, and Cabinet',
       'Amendment No. 4: Voter Approval of Constitutional Amendments',
       'Amendment No. 5: Limitations on Homestead Property Tax Assessments; increased portability period to transfer accrued benefit',
       'Amendment No. 6: Ad Valorem Tax Discount for Spouses of Certain Deceased Veterans Who Had Permanent, Combat-Related Disabilities']

In [100]:
contest_name_change_dict = {
'Total Registered Voters':'REG_VOTERS',
'Amendment No. 1: Citizenship Requirement to Vote in Florida Elections-No for Rejection-PARTY:NOP':'G20A01NO',
'Amendment No. 1: Citizenship Requirement to Vote in Florida Elections-Yes for Approval-PARTY:NOP':'G20A01YES',
'Amendment No. 2: Raising Florida’s Minimum Wage-No for Rejection-PARTY:NOP':'G20A02NO',
'Amendment No. 2: Raising Florida’s Minimum Wage-Yes for Approval-PARTY:NOP':'G20A02YES',
'Amendment No. 3: All Voters Vote in Primary Elections for State Legislature, Governor, and Cabinet-No for Rejection-PARTY:NOP- Governor, and Cabinet':'G20A03NO',
'Amendment No. 3: All Voters Vote in Primary Elections for State Legislature, Governor, and Cabinet-Yes for Approval-PARTY:NOP- Governor, and Cabinet':'G20A03YES',
'Amendment No. 4: Voter Approval of Constitutional Amendments-No for Rejection-PARTY:NOP':'G20A04NO',
'Amendment No. 4: Voter Approval of Constitutional Amendments-Yes for Approval-PARTY:NOP':'G20A04YES',
'Amendment No. 5: Limitations on Homestead Property Tax Assessments; increased portability period to transfer accrued benefit-No for Rejection-PARTY:NOP':'G20A05NO',
'Amendment No. 5: Limitations on Homestead Property Tax Assessments; increased portability period to transfer accrued benefit-Yes for Approval-PARTY:NOP':'G20A05YES',
'Amendment No. 6: Ad Valorem Tax Discount for Spouses of Certain Deceased Veterans Who Had Permanent, Combat-Related Disabilities-No for Rejection-PARTY:NOP- Combat-Related Disabilities':'G20A06NO',
'Amendment No. 6: Ad Valorem Tax Discount for Spouses of Certain Deceased Veterans Who Had Permanent, Combat-Related Disabilities-Yes for Approval-PARTY:NOP- Combat-Related Disabilities':'G20A06YES',
'President of the United States-Biden / Harris-PARTY:DEM':'G20PREDBID',
'President of the United States-Blankenship / Mohr-PARTY:CPF':'G20PRECBLA',
'President of the United States-De La Fuente / Richardson-PARTY:REF':'G20PREODEL',
'President of the United States-Hawkins / Walker-PARTY:GRE':'G20PREGHAW',
'President of the United States-Jorgensen / Cohen-PARTY:LPF':'G20PRELJOR',
'President of the United States-La Riva / Freeman-PARTY:PSL':'G20PRESLAR',
'President of the United States-Trump / Pence-PARTY:REP':'G20PRERTRU',
'President of the United States-WriteinVotes-PARTY:WRI':'G20PREOWRI',
'Retention of Carlos G. Muñiz-No-PARTY:NOP-Justice of the Supreme Court':'G20SSCNNO',
'Retention of Carlos G. Muñiz-Yes-PARTY:NOP-Justice of the Supreme Court':'G20SSCNYES'}

In [101]:
for contest in pivoted_2020.columns:
    if contest not in keep_names and "Amendment" not in contest and "Carlos G. Mu" not in contest and "President" not in contest:
        # Add in a condition about the 20
        value = "G"+ get_race(contest)+ get_party(contest)+ get_name(contest)
        contest_name_change_dict[contest] = value
    else:
        print("'"+contest+"':'',")


'pct_std':'',
'County Code (Three-character abbreviation)':'',
'County Name':'',
'Total Registered Voters':'',
'Amendment No. 1: Citizenship Requirement to Vote in Florida Elections-No for Rejection-PARTY:NOP':'',
'Amendment No. 1: Citizenship Requirement to Vote in Florida Elections-Yes for Approval-PARTY:NOP':'',
'Amendment No. 2: Raising Florida’s Minimum Wage-No for Rejection-PARTY:NOP':'',
'Amendment No. 2: Raising Florida’s Minimum Wage-Yes for Approval-PARTY:NOP':'',
'Amendment No. 3: All Voters Vote in Primary Elections for State Legislature, Governor, and Cabinet-No for Rejection-PARTY:NOP- Governor, and Cabinet':'',
'Amendment No. 3: All Voters Vote in Primary Elections for State Legislature, Governor, and Cabinet-Yes for Approval-PARTY:NOP- Governor, and Cabinet':'',
'Amendment No. 4: Voter Approval of Constitutional Amendments-No for Rejection-PARTY:NOP':'',
'Amendment No. 4: Voter Approval of Constitutional Amendments-Yes for Approval-PARTY:NOP':'',
'Amendment No. 5: Limit

In [102]:
contest_name_change_dict

{'Total Registered Voters': 'REG_VOTERS',
 'Amendment No. 1: Citizenship Requirement to Vote in Florida Elections-No for Rejection-PARTY:NOP': 'G20A01NO',
 'Amendment No. 1: Citizenship Requirement to Vote in Florida Elections-Yes for Approval-PARTY:NOP': 'G20A01YES',
 'Amendment No. 2: Raising Florida’s Minimum Wage-No for Rejection-PARTY:NOP': 'G20A02NO',
 'Amendment No. 2: Raising Florida’s Minimum Wage-Yes for Approval-PARTY:NOP': 'G20A02YES',
 'Amendment No. 3: All Voters Vote in Primary Elections for State Legislature, Governor, and Cabinet-No for Rejection-PARTY:NOP- Governor, and Cabinet': 'G20A03NO',
 'Amendment No. 3: All Voters Vote in Primary Elections for State Legislature, Governor, and Cabinet-Yes for Approval-PARTY:NOP- Governor, and Cabinet': 'G20A03YES',
 'Amendment No. 4: Voter Approval of Constitutional Amendments-No for Rejection-PARTY:NOP': 'G20A04NO',
 'Amendment No. 4: Voter Approval of Constitutional Amendments-Yes for Approval-PARTY:NOP': 'G20A04YES',
 'Amendm

In [103]:
for val in contest_name_change_dict.values():
    print(val)
    print(len(val))

REG_VOTERS
10
G20A01NO
8
G20A01YES
9
G20A02NO
8
G20A02YES
9
G20A03NO
8
G20A03YES
9
G20A04NO
8
G20A04YES
9
G20A05NO
8
G20A05YES
9
G20A06NO
8
G20A06YES
9
G20PREDBID
10
G20PRECBLA
10
G20PREODEL
10
G20PREGHAW
10
G20PRELJOR
10
G20PRESLAR
10
G20PRERTRU
10
G20PREOWRI
10
G20SSCNNO
9
G20SSCNYES
10
GCON03DCHR
10
GCON05DLAW
10
GCON15DCOH
10
GCON01NORA
10
GCON20DHAS
10
GCON17DELL
10
GCON13RLUN
10
GCON08RPOS
10
GCON18RMAS
10
GCON19RDON
10
GCON23RSPA
10
GCON26RGIM
10
GCON21NMAL
10
GCON13DCRI
10
GCON24NOLI
10
GCON14RQUI
10
GCON19DBAN
10
GCON06DCUR
10
GCON11DCOT
10
GCON11RWEB
10
GCON09DSOT
10
GCON26DMUC
10
GCON23DSCH
10
GCON04DDEE
10
GCON27DSHA
10
GCON24DWIL
10
GCON05RADL
10
GCON20RMUS
10
GCON17RSTE
10
GCON12RBIL
10
GCON22RPRU
10
GCON08DKEN
10
GCON04RRUT
10
GCON18NMIL
10
GCON03RCAM
10
GCON14DCAS
10
GCON12DWAL
10
GCON21RLOO
10
GCON24RSPI
10
GCON07RVAL
10
GCON21DFRA
10
GCON16DGOO
10
GCON27RSAL
10
GCON01RGAE
10
GCON06RWAL
10
GCON02RDUN
10
GCON18DKEI
10
GCON01DEHR
10
GCON15RFRA
10
GCON07DMUR
10
GCON22DDEU

In [104]:
pivoted_2020.rename(columns=contest_name_change_dict,inplace=True)

In [105]:
df = pd.DataFrame([(v, k) for k, v in contest_name_change_dict.items()], columns=['Candidate', 'Column'])
# Store the data into a csv file
#df.to_csv('./cand_dicts/oh_gen_20_st_prec.csv', sep=',')

In [106]:
pivoted_2020

,pct_std,County Code (Three-character abbreviation),County Name,REG_VOTERS,G20A01NO,G20A01YES,G20A02NO,G20A02YES,G20A03NO,G20A03YES,G20A04NO,G20A04YES,G20A05NO,G20A05YES,G20A06NO,G20A06YES,G20PREDBID,G20PRECBLA,G20PREODEL,G20PREGHAW,G20PRELJOR,G20PRESLAR,G20PRERTRU,G20PREOWRI,GCON03DCHR,GCON05DLAW,GCON15DCOH,GCON01NORA,GCON20DHAS,GCON17DELL,GCON13RLUN,GCON08RPOS,GCON18RMAS,GCON19RDON,GCON23RSPA,GCON26RGIM,GCON21NMAL,GCON13DCRI,GCON24NOLI,GCON14RQUI,GCON19DBAN,GCON06DCUR,GCON11DCOT,GCON11RWEB,GCON09DSOT,GCON26DMUC,GCON23DSCH,GCON04DDEE,GCON27DSHA,GCON24DWIL,GCON05RADL,GCON20RMUS,GCON17RSTE,GCON12RBIL,GCON22RPRU,GCON08DKEN,GCON04RRUT,GCON18NMIL,GCON03RCAM,GCON14DCAS,GCON12DWAL,GCON21RLOO,GCON24RSPI,GCON07RVAL,GCON21DFRA,GCON16DGOO,GCON27RSAL,GCON01RGAE,GCON06RWAL,GCON02RDUN,GCON18DKEI,GCON01DEHR,GCON15RFRA,GCON07DMUR,GCON22DDEU,GCON17NMUR,GCON10DDEM,GCON10RFRA,GCON16RBUC,GCON09ROLS,GCON07NGAR,GCON10OWRI,GCON13OWRI,GCON19OWRI,GCON02OWRI,GCON21OWRI,GCON23OWRI,GCON24OWRI,GCON27OWRI,GCON04OWRI,GCON06OWRI,GCON09OWRI,G20SSCNNO,G20SSCNYES,GSL076RBOT,GSL024DMOR,GSL054DGRI,GSL002RAND,GSL110RRIZ,GSL006DBAY,GSL009DTAN,GSL036RMAR,GSL059DLEA,GSL071DMEL,GSL062RCAP,GSL003DHOO,GSL047DESK,GSL110DCOL,GSL076DWEB,GSL118RROD,GSL032RSAB,GSL037RZIK,GSL042DCAD,GSL022DBYR,GSL068DDIA,GSL016DMAR,GSL035RING,GSL030RCOR,GSL116DLYN,GSL106RROM,GSL019RPAY,GSL033RHAG,GSL038DSTA,GSL112RBAR,GSL044RPOR,GSL111RAVI,GSL049DSMI,GSL093RLAM,GSL039DCAU,GSL067RLAT,GSL065RSPR,GSL010RBRA,GSL021RCLE,GSL103DPOL,GSL012RYAR,GSL058DTOW,GSL120DBAR,GSL040RBUR,GSL011RBYR,GSL031DSTI,GSL017RSTE,GSL023DROS,GSL048DMOR,GSL084RTRA,GSL116RPER,GSL036DEND,GSL088RMAD,GSL079DFOR,GSL017DROG,GSL105RBOR,GSL075DJON,GSL073DFAI,GSL087DSIL,GSL028RSMI,GSL067DDOU,GSL084DJOH,GSL114RCAB,GSL002DKRU,GSL027DGUZ,GSL072DBUC,GSL034DGOS,GSL083DBER,GSL082DACK,GSL026RFET,GSL091DSLO,GSL040NMIC,GSL012DBLI,GSL054RGRA,GSL072RMCF,GSL115DCES,GSL001DMAT,GSL042RHAW,GSL104RNAV,GSL044DTHO,GSL087RSEN,GSL119DMOH,GSL060RTOL,GSL120RMOO,GSL074RBUC,GSL056DDAV,GSL040DBAR,GSL041DWES,GSL016RFIS,GSL007RSHO,GSL006RTRU,GSL003RWIL,GSL114DJEA,GSL078RPER,GSL069DWEB,GSL047RSIS,GSL064DHAR,GSL048RMAR,GSL089DBON,GSL085DCAR,GSL009RKAL,GSL051DMAJ,GSL022RHAR,GSL004DPLA,GSL082RSNY,GSL025LHAN,GSL090DCAS,GSL011DHIC,GSL077DLOP,GSL039RTOM,GSL030DGOF,GSL119RFER,GSL029NROD,GSL060DJEN,GSL055RTUC,GSL021DENN,GSL035DLAU,GSL031RTRU,GSL065DJOH,GSL081DSKI,GSL018LWIL,GSL019DDUG,GSL080DNOV,GSL080RMEL,GSL062NROD,GSL058RMCC,GSL018DEDW,GSL042NSAN,GSL069RCHA,GSL093DGON,GSL055DTRI,GSL074DSTO,GSL052DDAB,GSL090RMAL,GSL033DMEL,GSL101DWOO,GSL068RTIT,GSL086DWIL,GSL105DPOR,GSL056RBEL,GSL028NRIC,GSL075RGRA,GSL059ROWE,GSL001RSAL,GSL057RBEL,GSL089RCAR,GSL077RGIA,GSL092NCLA,GSL112DDUR,GSL066RDIC,GSL050DYOA,GSL088DHAR,GSL028DBAK,GSL092DHAW,GSL066DPLA,GSL026DHEN,GSL004RMAN,GSL024RREN,GSL053DMOO,GSL034RMAS,GSL053RFIN,GSL038RMAG,GSL050RPLA,GSL085RROT,GSL118DJUN,GSL049RPRA,GSL104DBAR,GSL010DABO,GSL111DHAN,GSL088NAND,GSL018RGAR,GSL041RKIL,GSL106DMCF,GSL025DZUT,GSL081RBAN,GSL091RHUS,GSL057DHOT,GSL029RPLA,GSL078DWIL,GSL079RROA,GSL023RMCC,GSL032DDUK,GSL062DVAL,GSL086RRIV,GSL037DGAR,GSL015DTHO,GSL007DKHA,GSL052RALT,GSL083ROVE,GSL103RFAB,GSL025RLEE,GSL073RGRE,GSL029DKAG,GSL064RKOS,GSL051RSIR,GSL115RALO,GSL101RPAR,GSL027RBAR,GSL071RROB,GSL116OWRI,GSL004OWRI,GSL053OWRI,GSL055OWRI,GSL015RDUG,GSU37NROD,GSU39RROD,GSU21DELD,GSU29RNOR,GSU39NALF,GSU19NPAY,GSU25DROB,GSU20RBUR,GSU19DROU,GSU17RMAY,GSU01RBRO,GSU25RHAR,GSU07DHUN,GSU37RGAR,GSU09RBRO,GSU39DFER,GSU05RBRA,GSU09NIAN,GSU21RBOY,GSU23RGRU,GSU37DROD,GSU13RAND,GSU11RADA,GSU01DBUT,GSU23DNOR,GSU20DLEW,GSU13DSTE,GSU03DAUS,GSU31DBER,GSU15RMIN,GSU03RPRE,GSU15NJAM,GSU09DSIG,GSU17NSNY,GSU27DBRO,GSU11DBRA,GSU27RROD,GSU23NKAP,GSU17DFRE,GSU35DJON,GSU05DPET,GSU31RDON,GSU29DPOL,GSU07RHUT,GSU15DTOR,GSU03OWRI,GSU35OWRI,GSU07OWRI
0,ALA0001,ALA,Alachua,1540,231.0,898.0,637.0,496.0,543.0,572.0,624.0,482.0,352.0,742.0,158.0,955.0,424.0,0.0,2.0,1.0,6.0,1.0,725.0,1.0,426.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

## Check Statewide Totals

In [107]:
def statewide_totals_check(partner_df,source_df,column_list):
    """Compares the totals of two election result dataframes at the statewide total level

    Args:
      partner_df: DataFrame of election results we are comparing against
      source_df: DataFrame of election results we are comparing to
      column_list: List of races that there are votes for
 
    Returns:
      Nothing, only prints out an analysis
    """
    print("***Statewide Totals Check***")
    for race in column_list:
        if (partner_df[race].sum()- source_df[race].sum() != 0):
            print(race+" has a difference of "+str(partner_df[race].sum()-source_df[race].sum())+" votes")
            print("\tVEST: "+str(partner_df[race].sum())+" votes")
            print("\tSOURCES: "+str(source_df[race].sum())+" votes")
        else:
            print(race + " is equal", "\tVEST / RDH: " + str(partner_df[race].sum()))

In [108]:
statewide_totals_check(vest_fl_20,pivoted_2020,data_columns)

***Statewide Totals Check***
G20PRERTRU has a difference of 15.0 votes
	VEST: 5668731 votes
	SOURCES: 5668716.0 votes
G20PREDBID has a difference of 9.0 votes
	VEST: 5297045 votes
	SOURCES: 5297036.0 votes
G20PRELJOR has a difference of 1.0 votes
	VEST: 70324 votes
	SOURCES: 70323.0 votes
G20PREODEL is equal 	VEST / RDH: 5966
G20PRESLAR is equal 	VEST / RDH: 5712
G20PREGHAW is equal 	VEST / RDH: 14721
G20PRECBLA is equal 	VEST / RDH: 3902
G20PREOWRI is equal 	VEST / RDH: 24468


In [109]:
pivoted_2020.columns

Index(['pct_std', 'County Code (Three-character abbreviation)', 'County Name',
       'REG_VOTERS', 'G20A01NO', 'G20A01YES', 'G20A02NO', 'G20A02YES',
       'G20A03NO', 'G20A03YES',
       ...
       'GSU17DFRE', 'GSU35DJON', 'GSU05DPET', 'GSU31RDON', 'GSU29DPOL',
       'GSU07RHUT', 'GSU15DTOR', 'GSU03OWRI', 'GSU35OWRI', 'GSU07OWRI'],
      dtype='object', length=346)

## Check County-by-County Totals

In [110]:
#Add a county column
pivoted_2020["county"] = pivoted_2020["pct_std"].str[0:3]

In [111]:
def county_totals_check(partner_df,source_df,column_list,county_col,full_print=False):
    """Compares the totals of two election result dataframes at the county level

    Args:
      partner_df: DataFrame of election results we are comparing against
      source_df: DataFrame of election results we are comparing to
      column_list: List of races that there are votes for
      county_col: String of the column name that contains county information
      full_print: Boolean specifying whether to print out everything, including counties w/ similarities

    Returns:
      Nothing, only prints out an analysis
    """
    
    print("***Countywide Totals Check***")
    print("")
    diff_counties=[]
    for race in column_list:
        diff = partner_df.groupby([county_col]).sum()[race]-source_df.groupby([county_col]).sum()[race]
        for val in diff[diff != 0].index.values.tolist():
            if val not in diff_counties:
                diff_counties.append(val)
        if len(diff[diff != 0]!=0):   
            print(race + " contains differences in these counties:")
            for val in diff[diff != 0].index.values.tolist():
                county_differences = diff[diff != 0]
                print("\t"+val+" has a difference of "+str(county_differences[val])+" votes")
                print("\t\tVEST: "+str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
                print("\t\tSOURCES: "+str(source_df.groupby([county_col]).sum().loc[val,race])+" votes")
            if (full_print):
                for val in diff[diff == 0].index.values.tolist():
                    county_similarities = diff[diff == 0]
                    print("\t"+val + ": "+ str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
        else:
            print(race + " is equal across all counties")
            if (full_print):
                for val in diff[diff == 0].index.values.tolist():
                    county_similarities = diff[diff == 0]
                    print("\t"+val + ": "+ str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")

In [112]:
county_totals_check(vest_fl_20,pivoted_2020,data_columns,"county",full_print=False)

#Used the below to check against the official state totals
#vest_fl_20.groupby(["county"]).sum().to_csv("./vest_county_totals.csv")

***Countywide Totals Check***

G20PRERTRU contains differences in these counties:
	MON has a difference of 10.0 votes
		VEST: 25693 votes
		SOURCES: 25683.0 votes
	SEM has a difference of 5.0 votes
		VEST: 125241 votes
		SOURCES: 125236.0 votes
G20PREDBID contains differences in these counties:
	MON has a difference of 5.0 votes
		VEST: 21881 votes
		SOURCES: 21876.0 votes
	SEM has a difference of 4.0 votes
		VEST: 132528 votes
		SOURCES: 132524.0 votes
G20PRELJOR contains differences in these counties:
	MON has a difference of 1.0 votes
		VEST: 348 votes
		SOURCES: 347.0 votes
G20PREODEL is equal across all counties
G20PRESLAR is equal across all counties
G20PREGHAW is equal across all counties
G20PRECBLA is equal across all counties
G20PREOWRI is equal across all counties


### Precinct-by-Precinct Check

In [113]:
#Check if the columns are unique
print(vest_fl_20["pct_std"].value_counts(dropna=False))
print(pivoted_2020["pct_std"].value_counts(dropna=False))

GAD0009    1
SAR0429    1
DUV0912    1
HIL0817    1
PAL3174    1
          ..
CLL0428    1
PAL6045    1
PIN0213    1
BROV035    1
BRE0403    1
Name: pct_std, Length: 6010, dtype: int64
DAD0100    7
DAD0281    3
DAD0843    3
DAD0659    3
DAD0534    3
          ..
DAD0136    1
OSC0104    1
DUV0804    1
PAL5005    1
BRE0403    1
Name: pct_std, Length: 6014, dtype: int64


In [114]:
data_columns_full = [i for i in pivoted_2020.columns if i not in ['pct_std', 'County Code (Three-character abbreviation)', 'County Name','REG_VOTERS',"county"]]

In [115]:
pivoted_2020.loc[pivoted_2020["pct_std"]=="CHA54.1",'G20A01YES']

939    552.0
Name: G20A01YES, dtype: float64

In [116]:
pivoted_2020['GSU07OWRI']=="ALA"

0       False
1       False
2       False
3       False
4       False
        ...  
6090    False
6091    False
6092    False
6093    False
6094    False
Name: GSU07OWRI, Length: 6095, dtype: bool

In [117]:
#Combine results in these two precincts to match VEST
pivoted_2020 =pivoted_2020.fillna(0)
for i in data_columns_full:
    print(i)
    pivoted_2020[i] = pivoted_2020[i].astype(int)
    
    pivoted_2020.loc[pivoted_2020["pct_std"]=="CHA54.0",i]+=int(pivoted_2020.loc[pivoted_2020["pct_std"]=="CHA54.1",i])

#Check that it works
pivoted_2020.loc[pivoted_2020["pct_std"]=="CHA54.0"]

#Remove the precinct that the votes were added from
pivoted_2020 = pivoted_2020[pivoted_2020["pct_std"]!="CHA54.1"]

G20A01NO
G20A01YES
G20A02NO
G20A02YES
G20A03NO
G20A03YES
G20A04NO
G20A04YES
G20A05NO
G20A05YES
G20A06NO
G20A06YES
G20PREDBID
G20PRECBLA
G20PREODEL
G20PREGHAW
G20PRELJOR
G20PRESLAR
G20PRERTRU
G20PREOWRI
GCON03DCHR
GCON05DLAW
GCON15DCOH
GCON01NORA
GCON20DHAS
GCON17DELL
GCON13RLUN
GCON08RPOS
GCON18RMAS
GCON19RDON
GCON23RSPA
GCON26RGIM
GCON21NMAL
GCON13DCRI
GCON24NOLI
GCON14RQUI
GCON19DBAN
GCON06DCUR
GCON11DCOT
GCON11RWEB
GCON09DSOT
GCON26DMUC
GCON23DSCH
GCON04DDEE
GCON27DSHA
GCON24DWIL
GCON05RADL
GCON20RMUS
GCON17RSTE
GCON12RBIL
GCON22RPRU
GCON08DKEN
GCON04RRUT
GCON18NMIL
GCON03RCAM
GCON14DCAS
GCON12DWAL
GCON21RLOO
GCON24RSPI
GCON07RVAL
GCON21DFRA
GCON16DGOO
GCON27RSAL
GCON01RGAE
GCON06RWAL
GCON02RDUN
GCON18DKEI
GCON01DEHR
GCON15RFRA
GCON07DMUR
GCON22DDEU
GCON17NMUR
GCON10DDEM
GCON10RFRA
GCON16RBUC
GCON09ROLS
GCON07NGAR
GCON10OWRI
GCON13OWRI
GCON19OWRI
GCON02OWRI
GCON21OWRI
GCON23OWRI
GCON24OWRI
GCON27OWRI
GCON04OWRI
GCON06OWRI
GCON09OWRI
G20SSCNNO
G20SSCNYES
GSL076RBOT
GSL024DMOR
GSL054D

### Allocate votes

From VEST's documentation:

> Lake 108, Osceola 999, Palm Beach 8002, Seminole 900s, and Monroe's "Cumulative" precinct (the latter appears on the county's detailed results but not the DOS's precinct results file) don't have geography. Brevard 999, Broward Z073, Collier 450, Flagler 999, Hillborough 999, Leon 9000, Miami-Dade 100, and Pinellas 512 do have geography, but they are just the county election offices. Both groups represent some type of vote not assigned to a particular geography, like UOCAVA results, so these were distributed across the county by candidate proportional to the vote each precinct recorded.

In [118]:
to_allocate = ['FLA0999',
 'LAK0108',
 'OSC0999',
 'PAL8001',
 'PAL8002',
 'SEM0900',
 'SEM0901',
 'SEM0902',
 'SEM0903',
 'SEM0904',
 'SEM0905',
 'SEM0906',
 'SEM0907',
 'SEM0908',
 'SEM0909',
 'SEM0910',
'BRE0999','DAD0100','BROZ073','CHA0054','CLL0450','HIL0999','LEO9000','PIN0512']

In [119]:
allocating_votes = pivoted_2020[pivoted_2020["pct_std"].isin(to_allocate)]
receiving_votes = pivoted_2020[~pivoted_2020["pct_std"].isin(to_allocate)]

In [130]:
receiving_votes.shape

(6065, 347)

In [129]:
allocating_votes.shape

(29, 347)

In [120]:
def allocate_absentee(df_receiving_votes,df_allocating,column_list,col_allocating,allocating_to_all_empty_precs=False):
    """Allocates votes proportionally to precincts, usually by share of precinct-reported vote

    Args:
      df_receiving_votes: DataFrame with precinct-level votes
      df_allocating: DataFrame with the votes to allocate
      column_list: List of races that votes are being allocated for
      col_allocating: String referring to what level the allocation occurs at (most often county)
      allocating_to_all_empty_precs: Boolean for special case where all votes in df_receiving_votes are 0

    Returns:
      The precinct-level votes dataframe (df_receiving_votes) with the allocated votes
    """
    
    #Fill any n/a values with 0
    df_receiving_votes = df_receiving_votes.fillna(0)
    #Grab the original columns, so we can filter back down to them later
    original_cols = list(df_receiving_votes.columns)
    
    #Add in the "Total Votes column"
    if (allocating_to_all_empty_precs):
        #In cases where every vote is 0, need to set the Total_Votes equal to 1 for proportional allocation
        df_receiving_votes.loc[:,"Total_Votes"]=1
    else:
        df_receiving_votes.loc[:,"Total_Votes"]=0
        for race in column_list:
            df_receiving_votes.loc[:,"Total_Votes"]+=df_receiving_votes.loc[:,race]
    
    #Create the needed dataframes
    precinct_specific_totals = pd.DataFrame(df_receiving_votes.groupby([col_allocating]).sum())
    precinct_specific_totals.reset_index(drop=False,inplace=True)
    to_dole_out_totals = pd.DataFrame(df_allocating.groupby([col_allocating]).sum())
    to_dole_out_totals.reset_index(drop=False,inplace=True)
    
    #Add in total sum check
    sum_dataframe = pd.DataFrame(columns=precinct_specific_totals.columns)
    for i in column_list:
        total_votes = precinct_specific_totals.loc[:,i].sum()+to_dole_out_totals.loc[:,i].sum()
        sum_dataframe.at[0,i]=total_votes.astype(int)
    
    #Check the allocating to empty precincts code
    if (allocating_to_all_empty_precs):
        for i in column_list:
            if(sum(precinct_specific_totals[i])!=0):
                print("Allocating to all empty precincts parameter incorrect")
                break
    
    #Print out any instances where the allocation, as written, won't work
    special_allocation_needed = []
    for index, row in precinct_specific_totals.iterrows():
        for race in column_list:
            if (row[race]==0):
                race_district = row[col_allocating]
                if race_district in to_dole_out_totals[col_allocating].unique():
                    to_allocate = int(to_dole_out_totals.loc[to_dole_out_totals[col_allocating]==race_district][race])
                    if (to_allocate != 0):
                        special_allocation_needed.append([race_district,race])
                        if(row["Total_Votes"]==0):
                            precinct_specific_totals.loc[index,"Total_Votes"]=1
                            col_val = row[col_allocating]
                            df_receiving_votes.loc[df_receiving_votes[col_allocating]==col_val,"Total_Votes"]=1

    #Create some new columns for each of these races to deal with the allocation
    for race in column_list:
        add_var = race+"_add"
        rem_var = race+"_rem"
        floor_var = race+"_floor"
        df_receiving_votes.loc[:,add_var]=0.0
        df_receiving_votes.loc[:,rem_var]=0.0
        df_receiving_votes.loc[:,floor_var]=0.0

    #Iterate over the rows
    #Note this function iterates over the dataframe two times so the rounded vote totals match the totals to allocate
    for index, row in df_receiving_votes.iterrows():
        if row[col_allocating] in to_dole_out_totals[col_allocating].unique():
            for race in column_list:
                add_var = race+"_add"
                rem_var = race+"_rem"
                floor_var = race+"_floor"
                #Grab the district
                county_id = row[col_allocating]
                if [county_id,race] in special_allocation_needed:
                    #Get the denominator for the allocation - the summed "total votes" for precincts in that grouping
                    denom = precinct_specific_totals.loc[precinct_specific_totals[col_allocating]==county_id]["Total_Votes"]
                    #Get one of the numerators, how many districtwide votes to allocate
                    numer = to_dole_out_totals.loc[to_dole_out_totals[col_allocating]==county_id][race]
                    #Get the "total votes" for this particular precinct
                    val = df_receiving_votes.at[index,"Total_Votes"]
                    #Get the vote share, the precincts % of total precinct votes in the district times votes to allocate
                else:
                    #Get the denominator for the allocation (the precinct vote totals)
                    denom = precinct_specific_totals.loc[precinct_specific_totals[col_allocating]==county_id][race]
                    #Get one of the numerators, how many districtwide votes to allocate
                    numer = to_dole_out_totals.loc[to_dole_out_totals[col_allocating]==county_id][race]
                    #Get the vote totals for this race in this precinct
                    val = df_receiving_votes.at[index,race]
                    #Get the vote share, the precincts % of total precinct votes in the district times votes to allocate
                if ((float(denom)==0)):
                    vote_share = 0
                else:
                    vote_share = (float(val)/float(denom))*float(numer)
                df_receiving_votes.at[index,add_var] = vote_share
                #Take the decimal remainder of the allocation
                df_receiving_votes.at[index,rem_var] = vote_share%1
                #Take the floor of the allocation
                df_receiving_votes.at[index,floor_var] = np.floor(vote_share)

    #After the first pass through, get the sums of the races by district to assist in the rounding            
    first_allocation = pd.DataFrame(df_receiving_votes.groupby([col_allocating]).sum())

    #Now we want to iterate district by district to work on rounding
    county_list = list(to_dole_out_totals[col_allocating].unique()) 

    #Iterate over the district
    for county in county_list:
        for race in column_list:
            add_var = race+"_add"
            rem_var = race+"_rem"
            floor_var = race+"_floor"
            #County how many votes still need to be allocated (because we took the floor of all the initial allocations)
            to_go = int(np.round((int(to_dole_out_totals.loc[to_dole_out_totals[col_allocating]==county][race])-first_allocation.loc[first_allocation.index==county,floor_var])))
            #Grab the n precincts with the highest remainders and round these up, where n is the # of votes that still need to be allocated
            for index in df_receiving_votes.loc[df_receiving_votes[col_allocating]==county][rem_var].nlargest(to_go).index:
                df_receiving_votes.at[index,add_var] = np.ceil(df_receiving_votes.at[index,add_var])

    #Iterate over every race again
    for race in column_list:
        add_var = race+"_add"
        #Round every allocation down to not add fractional votes
        df_receiving_votes.loc[:,add_var]=np.floor(df_receiving_votes.loc[:,add_var])
        df_receiving_votes.loc[:,race]+=df_receiving_votes.loc[:,add_var]
        df_receiving_votes.loc[:,race] = df_receiving_votes.loc[:,race].astype(int)
        #Check to make sure all the votes have been allocated
        if ((sum_dataframe.loc[:,race].sum()-df_receiving_votes.loc[:,race].sum()!=0)):
            print("Some issue in allocating votes for:", i)
            
    #Filter down to original columns
    df_receiving_votes = df_receiving_votes[original_cols]

    return df_receiving_votes

In [121]:
#Perform the allocation
pivoted_2020 = allocate_absentee(receiving_votes,allocating_votes,data_columns,"county",allocating_to_all_empty_precs=False)

In [122]:
join_attempt_one = pd.merge(vest_fl_20,pivoted_2020,how="outer",on="pct_std",indicator=True)
join_attempt_one["_merge"].value_counts()

both          5993
left_only       92
right_only      72
Name: _merge, dtype: int64

In [123]:
# election_vest_id_changes = pd.read_csv("./election_vest_id_changes.csv")
# election_vest_id_changes_dict = dict(zip(election_vest_id_changes["election_ID"],election_vest_id_changes["vest_ID"]))

In [124]:
#Make changes to increase joins
election_vest_id_changes_dict = {'UNI1A & 1B': 'UNI001A', 'UNI2A & 2B': 'UNI002A', 'UNI3A & 3B': 'UNI003A', 'UNI4A & 4C': 'UNI004A', 'UNI5A & 5C': 'UNI005A', 'CHA01.0': 'CHA0001', 'CHA02.0': 'CHA0002', 'CHA03.0': 'CHA0003', 'CHA04.0': 'CHA0004', 'CHA05.0': 'CHA0005', 'CHA06.0': 'CHA0006', 'CHA07.0': 'CHA0007', 'CHA08.0': 'CHA0008', 'CHA09.0': 'CHA0009', 'CHA10.0': 'CHA0010', 'CHA11.0': 'CHA0011', 'CHA12.0': 'CHA0012', 'CHA13.0': 'CHA0013', 'CHA14.0': 'CHA0014', 'CHA15.0': 'CHA0015', 'CHA16.0': 'CHA0016', 'CHA17.0': 'CHA0017', 'CHA18.0': 'CHA0018', 'CHA19.0': 'CHA0019', 'CHA20.0': 'CHA0020', 'CHA21.0': 'CHA0021', 'CHA22.0': 'CHA0022', 'CHA23.0': 'CHA0023', 'CHA24.0': 'CHA0024', 'CHA25.0': 'CHA0025', 'CHA26.0': 'CHA0026', 'CHA27.0': 'CHA0027', 'CHA28.0': 'CHA0028', 'CHA29.0': 'CHA0029', 'CHA30.0': 'CHA0030', 'CHA31.0': 'CHA0031', 'CHA32.0': 'CHA0032', 'CHA33.0': 'CHA0033', 'CHA34.0': 'CHA0034', 'CHA35.0': 'CHA0035', 'CHA36.0': 'CHA0036', 'CHA38.0': 'CHA0038', 'CHA39.0': 'CHA0039', 'CHA41.0': 'CHA0041', 'CHA42.0': 'CHA0042', 'CHA43.0': 'CHA0043', 'CHA44.0': 'CHA0044', 'CHA45.0': 'CHA0045', 'CHA46.0': 'CHA0046', 'CHA47.0': 'CHA0047', 'CHA48.0': 'CHA0048', 'CHA49.0': 'CHA0049', 'CHA52.0': 'CHA0052', 'CHA53.0': 'CHA0053', 'CHA55.0': 'CHA0055', 'CHA56.0': 'CHA0056', 'CHA57.0': 'CHA0057', 'CHA58.0': 'CHA0058', 'CHA60.0': 'CHA0060', 'CHA61.0': 'CHA0061', 'CHA63.0': 'CHA0063', 'CHA64.0': 'CHA0064', 'CHA66.0': 'CHA0066', 'CHA67.0': 'CHA0067', 'CHA68.0': 'CHA0068', 'CHA69.0': 'CHA0069', 'CHA75.0': 'CHA0075', 'CHA76.0': 'CHA0076', 'CHA77.0': 'CHA0077', 'CHA78.0': 'CHA0078', 'CHA80.0': 'CHA0080', 'CHA54.0': 'CHA0054'}

#Implement the changes
pivoted_2020["pct_std"] = pivoted_2020["pct_std"].map(election_vest_id_changes_dict).fillna(pivoted_2020["pct_std"])

#Join again
join_attempt_two = pd.merge(vest_fl_20,pivoted_2020,how="outer",on="pct_std",indicator=True)
join_attempt_two["_merge"].value_counts()

both          6065
left_only       20
right_only       0
Name: _merge, dtype: int64

In [125]:
#Confirm that the unjoined precincts contain 0 votes
for i in data_columns:
    print(i)
    print(sum(join_attempt_two[join_attempt_two["_merge"]=="left_only"][i+"_x"]))

G20PRERTRU
0
G20PREDBID
0
G20PRELJOR
0
G20PREODEL
0
G20PRESLAR
0
G20PREGHAW
0
G20PRECBLA
0
G20PREOWRI
0


In [126]:
def precinct_votes_check(merged_df,column_list,vest_on_left,name_col,print_level=0):
    """Checks a merged dataframe with two election results at the precinct level

    Args:
      merged_df: DataFrame with one set of election results joined to another
      column_list: List of races that there are votes for
      vest_on_left: Boolean specifying whether VEST data is on the left side of merged_df
      name_col: String of the column name to refer to precincts when a difference occurs
      print_level: Integer that specifies how large the vote difference in a precinct must be to be printed

    Returns:
      Nothing, only prints out an analysis
    """
    merged_df = merged_df.sort_values(by=[name_col],inplace=False)
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    diff_values = []
    max_diff = 0
    for index,row in merged_df.iterrows():
        same = True
        for i in column_list:
            left_data = i + "_x"
            right_data = i + "_y"
            if ((row[left_data] is None) or (row[right_data] is None) or (np.isnan(row[right_data])or(np.isnan(row[left_data])))):
                print("FIX NaN value at: ", row[name_col])
                return;
            diff = abs(row[left_data]-row[right_data])
            if (diff>0):
                same = False
                diff_values.append(abs(diff))
                if (diff>max_diff):
                    max_diff = diff
            if(diff>print_level):
                if (vest_on_left):
                    print(i, "{:.>72}".format(row[name_col]), "(V)","{:.>5}".format(int(row[left_data]))," (S){:.>5}".format(int(row[right_data])),"(D):{:>5}".format(int(row[left_data]-row[right_data])))                           
                else:
                    print(i, "{:.>72}".format(row[name_col]), "(S)","{:.>5}".format(int(row[left_data]))," (V){:.>5}".format(int(row[right_data])),"(D):{:>5}".format(int(row[left_data]-row[right_data])))
        if(same != True):
            different_rows +=1
            diff_list.append(row[name_col])
        else:
            matching_rows +=1
    print("")
    print("There are ", len(merged_df.index)," total rows")
    print(different_rows," of these rows have election result differences")
    print(matching_rows," of these rows are the same")
    print("")
    print("The max difference between any one shared column in a row is: ", max_diff)
    if(len(diff_values)!=0):
        print("The average difference is: ", str(sum(diff_values)/len(diff_values)))
    count_big_diff = len([i for i in diff_values if i > 10])
    print("There are ", str(count_big_diff), "precinct results with a difference greater than 10")
    print("")
    print("All precincts containing differences:")
    diff_list.sort()
    print(diff_list)

In [127]:
precinct_votes_check(join_attempt_two[join_attempt_two["_merge"]=="both"],data_columns,True,"pct_std",print_level=0)

G20PRERTRU .................................................................DAD0107 (V) ..294  (S)....4 (D):  290
G20PREDBID .................................................................DAD0107 (V) ..483  (S)....3 (D):  480
G20PRELJOR .................................................................DAD0107 (V) ....4  (S)....0 (D):    4
G20PRERTRU .................................................................DAD0107 (V) ..294  (S)..290 (D):    4
G20PREDBID .................................................................DAD0107 (V) ..483  (S)..480 (D):    3
G20PRERTRU .................................................................DAD0110 (V) ..522  (S)....0 (D):  522
G20PREDBID .................................................................DAD0110 (V) ..823  (S)....0 (D):  823
G20PRELJOR .................................................................DAD0110 (V) ....1  (S)....0 (D):    1
G20PREODEL .................................................................DAD0110 (V) 

G20PREGHAW .................................................................DAD0531 (V) ....5  (S)....4 (D):    1
G20PRERTRU .................................................................DAD0533 (V) ..275  (S)..137 (D):  138
G20PREDBID .................................................................DAD0533 (V) .1861  (S)..477 (D): 1384
G20PRELJOR .................................................................DAD0533 (V) ....3  (S)....1 (D):    2
G20PREODEL .................................................................DAD0533 (V) ....3  (S)....1 (D):    2
G20PRESLAR .................................................................DAD0533 (V) ....2  (S)....1 (D):    1
G20PREGHAW .................................................................DAD0533 (V) ....2  (S)....0 (D):    2
G20PRECBLA .................................................................DAD0533 (V) ....1  (S)....0 (D):    1
G20PREOWRI .................................................................DAD0533 (V) 

G20PRERTRU .................................................................MON0005 (V) .1000  (S)..999 (D):    1
G20PREDBID .................................................................MON0005 (V) .1706  (S).1705 (D):    1
G20PRELJOR .................................................................MON0005 (V) ...35  (S)...34 (D):    1
G20PREDBID .................................................................MON0006 (V) ..927  (S)..926 (D):    1
G20PREDBID .................................................................MON0008 (V) ..931  (S)..930 (D):    1
G20PREDBID .................................................................MON0010 (V) ..959  (S)..958 (D):    1
G20PRERTRU .................................................................MON0011 (V) .1213  (S).1212 (D):    1
G20PREDBID .................................................................MON0011 (V) .1101  (S).1100 (D):    1
G20PRERTRU .................................................................MON0012 (V) 